# 1. Idea Generation

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import sys
import sqlalchemy
from ipywidgets import *
import ipywidgets as widgets
from IPython.display import display
from traitlets import link

## 1.1 Data Prep

In [2]:
#Loading database after selection
## Connecting database
db_conn = sqlite3.connect('AVA_FY.db')
db_cur = db_conn.cursor()

In [3]:
#load all tables
db_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
metric_tables = db_cur.fetchall()
metric_names = [i[0] for i in metric_tables]
var_names = [name.replace(" ", "_") for name in metric_names]
df_ava = {}
for i in range(len(var_names)):
    sql_query = 'SELECT * FROM "{table_name}"'.format(table_name=metric_names[i])
    key = metric_names[i]
    value = pd.read_sql(sql_query, db_conn)
    df_ava[key] = value

In [4]:
criteria_table = {"Falling Revenue":"Total Revenue", 
                  "Rising COGS%":"COGS Ratio", 
                  "Falling Gross Profit Margin":"Gross Profit Margin", 
                  "Rising SG&A%":"SG&A Ratio",
                  "Falling EBT Margin":"EBT Margin", 
                  "Falling EBIT Margin":"EBIT Margin", 
                  "Falling EBITDA Margin":"EBITDA Margin", 
                  "Falling NOPAT Margin":"NOPAT Margin",
                  "Falling Inventory Turnover": "Inventory Turnover", 
                  "Falling Fixed Asset Turnover": "Fixed Asset Turnover", 
                  "Rising DSO":"DSO", 
                  "Falling DPO":"DPO", 
                 }
column_names = list(df_ava["Total Revenue"].columns)

## 1.2 Select Metrics, Criteria and Years to Generate Idea/Target Lists

In [5]:
falling_list = ["Falling Revenue", "Falling Gross Profit Margin","Falling EBT Margin", "Falling EBIT Margin", "Falling EBITDA Margin", "Falling NOPAT Margin",
                 "Falling Inventory Turnover", "Falling Fixed Asset Turnover", "Falling DSO",]
rising_list = ["Rising COGS%", "Rising SG&A%", "Rising DPO",]
year_list = column_names[column_names.index("FY1998"):]
select_ideas = widgets.Accordion(
    children=[widgets.SelectMultiple(description="Select Falling Metrics",options=falling_list),
              widgets.SelectMultiple(description="Select Rising Metrics",options=rising_list),
              widgets.SelectMultiple(description="Select Years To Be Included",options=year_list),
              widgets.FloatSlider(
                    value=0,
                    min=0,
                    max=2.0,
                    step=0.01,
                    description='Criteria:',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='.0%',
                    slider_color='#CB1B45'
                ),
             ])
select_ideas.set_title(0, 'Falling Metrics')
select_ideas.set_title(1, 'Rising Metrics')
select_ideas.set_title(2, 'Years')
select_ideas.set_title(3, 'Criteria (Inclusive WIP)')
select_ideas

In [106]:
column_names = list(df_ava["Total Revenue"].columns)

falling_list = list(select_ideas.children[0].value)
rising_list = list(select_ideas.children[1].value)
change_criteria = select_ideas.children[3].value

In [107]:
year_list = list(select_ideas.children[2].value)
inspection_years = year_list
year_len = len(inspection_years)
first_inspect_year_index = column_names.index(inspection_years[0])
inspection_year_previous = column_names[first_inspect_year_index-1:first_inspect_year_index+year_len-1]

In [108]:
## Define columns to display
info_list=["Company Name", "Exchange:Ticker", "Country", "Region", "AV Industry", "AV Sector"]
display_columns = info_list + inspection_years

## Falling metrics generation
temp_db = {}
temp_df = df_ava["Total Revenue"]
#Falling metrics
for idea in falling_list:
    table_name = criteria_table[idea]
    key=idea
    temp_df = df_ava[table_name]
    for i,j in zip(inspection_years, inspection_year_previous):
        temp_df = temp_df[temp_df.loc[ : , i] < temp_df.loc[ : , j]]
#         temp_df = temp_df[((temp_df.loc[:,i] - temp_df.loc[:,j]) / temp_df.loc[:,j]) < change_criteria]
    temp_db[key] = temp_df.loc[:,display_columns]

#Rising metrices
for idea in rising_list:
    table_name = criteria_table[idea]
    key=idea
    temp_df = df_ava[table_name]
    for i,j in zip(inspection_years, inspection_year_previous):
        temp_df = temp_df[temp_df.loc[ : , i] > temp_df.loc[ : , j]]
    temp_db[key] = temp_df.loc[:,display_columns]

In [109]:
temp_db.keys()

dict_keys(['Falling Revenue', 'Falling EBIT Margin', 'Falling NOPAT Margin', 'Rising COGS%', 'Rising SG&A%'])

In [110]:
for key in list(temp_db.keys()):
    print(temp_db[key].head())

                                  Company Name Exchange:Ticker        Country  \
3          Royal Dutch Shell plc (ENXTAM:RDSA)     ENXTAM:RDSA    Netherlands   
6           Exxon Mobil Corporation (NYSE:XOM)        NYSE:XOM  United States   
37              Chevron Corporation (NYSE:CVX)        NYSE:CVX  United States   
92   Archer-Daniels-Midland Company (NYSE:ADM)        NYSE:ADM  United States   
103                  ArcelorMittal (ENXTAM:MT)       ENXTAM:MT     Luxembourg   

            Region                 AV Industry  AV Sector  FY2013  FY2014  \
3           Europe  Resources/Process Material  Oil & Gas  451235  421105   
6    North America  Resources/Process Material  Oil & Gas  390247  364763   
37   North America  Resources/Process Material  Oil & Gas  211665  192308   
92   North America  Resources/Process Material       Food   89804   81201   
103         Europe  Resources/Process Material      Steel   79440   79282   

     FY2015  
3    264960  
6    236810  
37   122

In [121]:
temp_db['Falling Revenue']

,Company Name,Exchange:Ticker,Country,Region,AV Industry,AV Sector,FY2013,FY2014,FY2015
3,Royal Dutch Shell plc (ENXTAM:RDSA),ENXTAM:RDSA,Netherlands,Europe,Resources/Process Material,Oil & Gas,451235,421105,264960
6,Exxon Mobil Corporation (NYSE:XOM),NYSE:XOM,United States,North America,Resources/Process Material,Oil & Gas,390247,364763,236810
37,Chevron Corporation (NYSE:CVX),NYSE:CVX,United States,North America,Resources/Process Material,Oil & Gas,211665,192308,122566
92,Archer-Daniels-Midland Company (NYSE:ADM),NYSE:ADM,United States,North America,Resources/Process Material,Food,89804,81201,67702
103,ArcelorMittal (ENXTAM:MT),ENXTAM:MT,Luxembourg,Europe,Resources/Process Material,Steel,79440,79282,63578
113,Pfizer Inc. (NYSE:PFE),NYSE:PFE,United States,North America,Resources/Process Material,Pharma,51584,49605,48851
121,"Telefónica, S.A. (BME:TEF)",BME:TEF,Spain,Europe,TIME,Telecome Operators,60867.96424,53814.83431,51012.75239
126,Hewlett Packard Enterprise Company (NYSE:HPE),NYSE:HPE,United States,North America,TIME,Tech HW Equipment,57371,55123,52107
136,HP Inc. (NYSE:HPQ),NYSE:HPQ,United States,North America,TIME,Tech HW Equipment,112298,111454,103355
142,Statoil ASA (OB:STL),OB:STL,Norway,Europe,Resources/Process Material,Oil & Gas,73165.8059,71665.87735,54106.91589


In [56]:
#Find the suckers that appear in all lists
sucker_bag=[]
for idea in list(temp_db.keys()):
    sucker_bag = sucker_bag + list(temp_db[idea]["Company Name"])
len(sucker_bag)

1008

In [59]:
(set(sucker_bag))

{'ABB Ltd. (SWX:ABBN)',
 'ABC-Mart, Inc. (TSE:2670)',
 'ADTRAN, Inc. (NasdaqGS:ADTN)',
 'AF Gruppen ASA (OB:AFG)',
 'ALBIS Co., Ltd. (TSE:7475)',
 'AP (Thailand) Public Company Limited (SET:AP)',
 'ASKUL Corporation (TSE:2678)',
 'AT&T Inc. (NYSE:T)',
 'Abercrombie & Fitch Co. (NYSE:ANF)',
 'Abertis Infraestructuras S.A. (BME:ABE)',
 'Abu Dhabi Aviation (ADX:ADAVIATION)',
 'AcadeMedia AB (OM:ACAD)',
 'Accell Group N.V. (ENXTAM:ACCEL)',
 'Achilles Corporation (TSE:5142)',
 'Acom Co., Ltd. (TSE:8572)',
 'Adcorp Holdings Limited (JSE:ADR)',
 'Advantech Co., Ltd. (TSEC:2395)',
 'Affinion Group Holdings, Inc. (OTCPK:AFGR)',
 'Ajinomoto Co., Inc. (TSE:2802)',
 'Akka Technologies (ENXTPA:AKA)',
 'Al Ezz Dekheila Steel Company - Alexandria S.A.E. (CASE:IRAX)',
 'Al Rajhi Banking and Investment Corporation (SASE:1120)',
 'Alexander Forbes Group Holdings Limited (JSE:AFH)',
 'Alfa Laval AB (publ) (OM:ALFA)',
 'Alfresa Holdings Corporation (TSE:2784)',
 'Alliance Healthcare Services, Inc. (Nasdaq

## 1.3 Filter out Industries of our Interests

In [111]:
sector_list=list(set(df_ava["Total Revenue"]["AV Sector"]))
sector_list=list(np.sort(sector_list))
country_list=list(set(df_ava["Total Revenue"]["Country"]))
country_list=list(np.sort(country_list))

In [112]:
filter_idea_data = widgets.Accordion(
    children=[widgets.Dropdown(description="Select Sector",options=sector_list),
              widgets.Dropdown(description="Select Country",options=country_list),
             ])
filter_idea_data.set_title(0, 'Select Sector')
filter_idea_data.set_title(1, 'Select Country')
filter_idea_data

In [117]:
filter_button = widgets.Button(
    description='Load Filtered Data',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
display(filter_button)

def filter_button_click(click):
    global temp_db_filtered
    temp_db_filtered={}
    for table in list(temp_db.keys()):
        key=table
        temp_df = temp_db[table][temp_db[table]["Country"] == filter_idea_data.children[1].value]
#         temp_df = temp_df[temp_df["AV Sector"] == filter_idea_data.children[0].value]
        temp_db_filtered[key]=temp_df
    return(temp_db_filtered)
    print("Database Filtered by"+filter_idea_data.children[1].value)
    
filter_button.on_click(filter_button_click)

In [118]:
for idea in list(temp_db_filtered.keys()):
    print(temp_db_filtered[idea].head())

                                      Company Name Exchange:Ticker  \
6               Exxon Mobil Corporation (NYSE:XOM)        NYSE:XOM   
37                  Chevron Corporation (NYSE:CVX)        NYSE:CVX   
92       Archer-Daniels-Midland Company (NYSE:ADM)        NYSE:ADM   
113                         Pfizer Inc. (NYSE:PFE)        NYSE:PFE   
126  Hewlett Packard Enterprise Company (NYSE:HPE)        NYSE:HPE   

           Country         Region                 AV Industry  \
6    United States  North America  Resources/Process Material   
37   United States  North America  Resources/Process Material   
92   United States  North America  Resources/Process Material   
113  United States  North America  Resources/Process Material   
126  United States  North America                        TIME   

             AV Sector  FY2013  FY2014  FY2015  
6            Oil & Gas  390247  364763  236810  
37           Oil & Gas  211665  192308  122566  
92                Food   89804   81201   

In [119]:
#Find the suckers that appear in all lists
sucker_bag=[]
for idea in list(temp_db_filtered.keys()):
    sucker_bag = sucker_bag + list(temp_db_filtered[idea]["Company Name"])
sucker_bag = list(np.sort(list(set(sucker_bag))))

In [120]:
sucker_bag

['3D Systems Corporation (NYSE:DDD)',
 'A. Schulman, Inc. (NasdaqGS:SHLM)',
 'A.M. Castle & Co. (OTCPK:CASL)',
 'AAR Corp. (NYSE:AIR)',
 'ACCO Brands Corporation (NYSE:ACCO)',
 'ACI Worldwide, Inc. (NasdaqGS:ACIW)',
 'ADTRAN, Inc. (NasdaqGS:ADTN)',
 'AMCON Distributing Company (AMEX:DIT)',
 'ARRIS International plc (NasdaqGS:ARRS)',
 'Actuant Corporation (NYSE:ATU)',
 'Acxiom Corporation (NasdaqGS:ACXM)',
 'Adams Resources & Energy, Inc. (AMEX:AE)',
 'Adobe Systems Incorporated (NasdaqGS:ADBE)',
 'Aegion Corporation (NasdaqGS:AEGN)',
 'Affinion Group Holdings, Inc. (OTCPK:AFGR)',
 'Aflac Incorporated (NYSE:AFL)',
 'Agilent Technologies, Inc. (NYSE:A)',
 'Albany International Corp. (NYSE:AIN)',
 'Alere Inc. (NYSE:ALR)',
 'Alliance Data Systems Corporation (NYSE:ADS)',
 'Alliance Healthcare Services, Inc. (NasdaqGM:AIQ)',
 'Alon USA Energy, Inc. (NYSE:ALJ)',
 'Altra Industrial Motion Corp. (NasdaqGS:AIMC)',
 'Amazon.com, Inc. (NasdaqGS:AMZN)',
 'American Eagle Outfitters, Inc. (NYSE:AEO)